In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
sns.set(font_scale=1)

import warnings
warnings.filterwarnings('ignore')

In [2]:
def to_tokenize(text_Series, dict_change_word): # input은 '기사내용'을 pd.Series 타입으로 줌. out도 Series 타입.
    import re

    # 1. URL 부분 제거.
    print("\nURL 제거 과정.\n")
    text_Series = [re.sub('((http(s?))\:\/\/)([0-9a-zA-Z\-]+\.*\/*(\\n)*)+|url|URL', '', text) for text in tqdm(text_Series)]

    # 2. e-mail 주소 제거.
    print("\ne-mail 제거 과정.\n")
    text_Series = [re.sub('(\<.[a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4})', '', text) for text in tqdm(text_Series)] 

    # 3. 고유어 변경
    print("\n고유어 변경 과정.\n")
    #text_Series = text_Series.apply(lambda x: change_hiphen_word(x, dict_change_word))
    text_Series = [change_hiphen_word(text, dict_change_word) for text in tqdm(text_Series)]
    
    # 4. 특수문자 제거. 모두 제거.
    print("\n특수문자 제거 과정.\n")
    text_Series = [re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', text) for text in tqdm(text_Series)]

    # 5. 문자 외 모두 제거(공백, \, 숫자 사라짐)
    print("\n문자 외 제거 과정.\n")
    text_Series = [re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', text) for text in tqdm(text_Series)]

    return text_Series


In [3]:
#dict_change_word에 해당하는 단어를 변경하는 함수

def change_hiphen_word(text, dictionary):
    com_list = [".replace('{}', '{}')".format(word, dictionary[word]) for word in dictionary.keys() if word in text]
    com_list.insert(0, "'{}'.format(text)")
    command = ''.join(com_list)
    text = eval(command)

    return text

In [4]:
#Corpus 만드는 함수 (딕셔너리 반영)
def make_corpus(text_col):
    from gensim import corpora
    
    dictionary = pd.read_pickle('dictionary.pkl')
    dictionary.filter_extremes(no_below = 20)
    corpus = [dictionary.doc2bow(text) for text in text_col]
    
    return dictionary, corpus

In [128]:
#하나로 합쳐진 데이터 불러오기

news_entire = pd.read_csv('musinsa2.csv')

news_company = []

news_list = ['CSR','과징금','공정위','지역사회','사회공헌',
             '스타트업','청년','창업','중소기업','상생','협력사','갑질',
             '봉사','기부','ESG','사망','사고','취업','과태료','노조','개인정보']

for i in news_list:
    news_company.append(news_entire[news_entire.contents.str.contains(i) == True])

news_company = pd.concat(news_company)    

news_company = news_company.drop_duplicates(['contents','company'])

In [129]:
#딕셔너리 통합 (고유어들을 하나의 주제로 분류할 수 있게끔 설정)

dict_change_word= {'경북':'지역사회','광주':'지역사회','대구' : '지역사회' 
                   ,'외국인':'취약계층', '장애인':'취약계층', '공정거래위원회':'공정위',
                   '사회적 책임':'사회공헌','CSR':'사회공헌', '고객정보':'개인정보',
                   'ESG':'지속가능', '이노베이션':'혁신',
                   '동반성장' : '상생', '갑질' : '불공정', '甲' : '갑',
                   '산재':'산업재해', '산업 재해' : '산업재해'}

In [130]:
#토큰화하기 위해 데이터클리닝을 해줌

news_entire_to_tokenize = pd.DataFrame(to_tokenize(news_company['title'], dict_change_word))

news_entire_to_tokenize.reset_index(drop=True, inplace=True)

news_entire_to_tokenize.head(10)

100%|████████████████████████████████████████████████████████████████████████████| 623/623 [00:00<00:00, 249028.06it/s]


URL 제거 과정.


e-mail 제거 과정.


고유어 변경 과정.


특수문자 제거 과정.


문자 외 제거 과정.



,0
0,회원분쟁 나 몰라라 네이버 크림 등 리셀업체 불공정약관 개선
1,쿠팡 등 대형 유통업체 연말까지 납품업체에 천억원 지원
2,착취의 테크놀로지인가 플랫폼 죽이기인가
3,온라인 플랫폼법 제정 미리 불공정 한 무신사 공정위 불공정거래 해당
4,사업자 책임 면제하고 공정위 리셀 업체 불공정약관 시정
5,백화점 마트 아울렛 수수료인하 코로나 상생
6,무신사 중소 패션 브랜드 상생 지원금 억 돌파
7,무신사 스타일쉐어 인수 본계약 체결 인수대금 억 원
8,네이버 크림 등 한정판 플랫폼 짝퉁 거래 무조건 면책 안 된다
9,공정위 서비스 임의 변경 등 리셀 업체 불공정 약관 시정


In [131]:
#Stopword에 추가할 리스트 (조사, 명사, 일부 동사)

stopword =  pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt")['아'].tolist()

stopwords_post = ['이', '가', '께서', '에서', '서', '이다', '의', '을', '를', '이', '가', '에', '에게', '께', '한테', '에서', '에게서', '로서', '로써', '고', '이라고', '라고', '와', '과', '이랑', '에', '같이', '처럼', '만큼', '만치', '야',
                  '아', '여', '이여', '이시여', '와', '과', '하고', '이다', '이며', '에다', '에다가', '이랑', '랑']

my_stopwords = pd.read_table("stopword_list.txt")['목록'].values.tolist()

stopword = set(stopword + stopwords_post + my_stopwords)

In [132]:
#Tagging 작업

from konlpy.tag import Okt

okt = Okt()

news_tagged = news_entire_to_tokenize[0].apply(lambda x : okt.morphs(x, stem = True)).tolist()

#불용어 제거

news_tagged = [[word for word in word_list if word not in stopword] for word_list in tqdm(news_tagged)]

#한글자 제거

news_tagged = [[word for word in word_list if len(word) > 1] for word_list in tqdm(news_tagged)]

100%|████████████████████████████████████████████████████████████████████████████| 623/623 [00:00<00:00, 623788.83it/s]


In [133]:
#태깅 완료 후 제목을 태깅한 것으로 대체해줌

news_ready_for_LDA = news_company
news_ready_for_LDA['title'] = news_tagged

news_ready_for_LDA.to_excel('dddddd.xlsx')

In [134]:
#데이터 클리닝

news_ready_for_LDA.drop(news_ready_for_LDA.loc[news_ready_for_LDA['date']=='CHIEF EXECUTIVE'].index, inplace=True)
news_ready_for_LDA.drop(news_ready_for_LDA.loc[news_ready_for_LDA['date']=='date'].index, inplace=True)
news_ready_for_LDA['date'] = pd.to_datetime(news_ready_for_LDA['date'], format = "%Y-%m-%d")

In [135]:
news_ready_for_LDA.reset_index(drop=True, inplace=True)

news_ready_for_LDA

,Column1,date,title,contents,company
0,952.0,2021-11-28,"[회원, 분쟁, 모르다, 크림, 리셀, 업체, 불공정, 약관, 개선]","공정위, 5개 업체에 약관조항 시정 조현미 기자 hmcho@ajunews.com 무...",무신사
1,1935.0,2021-02-02,"[쿠팡, 대형, 유통업체, 연말, 납품, 업체, 천억원, 지원]","(쿠팡·SSG·마켓컬리·무신사), 아울렛 4곳(롯데·현대·신세계·뉴코아아울렛) 등 ...",무신사
2,2695.0,2021-10-12,"[착취, 테크놀로지, 인가, 이기, 인가]","이 밖에도 숙박 플랫폼 야놀자나 패션 플랫폼 무신사, 네이버 등도 비슷한 문제로 고...",무신사
3,810.0,2021-03-07,"[온라인, 제정, 미리, 불공정, 물다, 공정위, 불공정, 거래, 해당]","공정위 ""무신사, 공정거래법 상 경쟁 제한 위반 소지있어"" 무신사, 온라인 플랫폼 ...",무신사
4,950.0,2021-11-28,"[사업자, 책임, 면제, 공정위, 리셀, 업체, 불공정, 약관, 시정]","공정위는 ""그동안 개인의 거래나 중소플랫폼 위주였던 리셀 시장에 네이버 계열사가 운...",무신사
...,...,...,...,...,...
618,1551.0,2021-11-10,"[개인정보, 유출, 개사, 과태료, 만원]",‘개인정보 안전성 확보를 위한 조처’를 소홀히 해 개인정보가 유출된 GS리테일과 무...,무신사
619,1554.0,2021-11-10,"[알파, 천명, 고객, 전화번호, 외부, 노출]","유출된 무신사, 위버스컴퍼니, 동아오츠카, 한국신용데이터, 디엘이앤씨, GS리테일,...",무신사
620,1528.0,2021-11-10,"[물다, 동아오츠카, 제재, 처분]",▶GS25 편의점 등 유통 기업 ‘GS리테일’ ▶건설사 ‘DL이앤씨’(옛 대림산업)...,무신사
621,1536.0,2021-11-10,"[물다, 개인정보, 유출, 과태료, 처분]",ⓒ뉴시스 개인정보 보호를 위한 안전조치에 필요한 의무를 다하지 않아 이용자의 정보가...,무신사


In [136]:
from gensim import corpora, models

#ldamodel = models.wrappers.LdaMallet.load(datapath("model"))

ldamodel = pd.read_pickle('lda_model.pkl')

dictionary, corpus = make_corpus(news_ready_for_LDA['title'])

In [137]:
#토픽을 각 열로 옮김

topic_list = []

for i, j in enumerate(ldamodel[corpus]):
    prob_list = []
    temp = j
    
    for k, (topic_num, prop_topic) in enumerate(temp):
        prob_list.append(prop_topic)
    
    
    prob_df = pd.DataFrame(prob_list)
    
    prob_df_after_choice = [prob_df == max(prob_list)][0].T.astype(int)
    
    prob_df_after_choice = prob_df_after_choice / sum(prob_df_after_choice.T[0])
    
    topic_list.append(prob_df_after_choice)
    

In [138]:
topic_df = pd.concat(topic_list)
topic_df.reset_index(drop=True, inplace=True)

topic_df.to_excel('dd.xlsx')

In [139]:
news_topic_df = pd.concat([news_ready_for_LDA['date'], topic_df, news_ready_for_LDA['company']],axis=1)

In [140]:
#각 날짜에 해당하는 모든 기사 Topic 정보 합치는 함수
def grouped_by_date(concat_df, num_topics):
    grouped_df = concat_df[[x for x in range(num_topics)]].groupby(concat_df['year'])
    
    grouped_with_sum = grouped_df.sum()
    
    grouped_with_sum['합계'] = grouped_with_sum.sum(axis=1)
    
    return grouped_with_sum

In [141]:
def normalized_topic(topic_sum_df):
    topic_sum_df2 = topic_sum_df.iloc[0:,0:9]
    for i in range(len(topic_sum_df2)):
        topic_sum_df2.iloc[i] = topic_sum_df2.iloc[i] / topic_sum_df2.iloc[i, -1]
        
    topic_sum_df2['company'] = topic_sum_df['company']
    return topic_sum_df2

In [142]:
news_topic_df = news_topic_df.dropna()

news_topic_df['year'] = news_topic_df['date'].dt.year
##def 실행 - 날짜 기준으로 기사의 topic 정보 합침
    
news_topic_normalized = grouped_by_date(news_topic_df, 8)
news_topic_normalized['company'] = news_topic_df['company'].iloc[0]

news_topic_normalized = normalized_topic(news_topic_normalized)

news_topic_normalized.reset_index(drop=False, inplace=True)


In [143]:
# topic 컬럼 정리

columns = ["date"] + ["Topic_{}".format(i) for i in range(8)] + ['합계'] + ['company']
news_topic_normalized.columns = columns

In [144]:
news_topic_normalized

,date,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,합계,company
0,2021,0.05,0.181461,0.119395,0.146442,0.209952,0.06664,0.148181,0.077929,1.0,무신사


In [145]:
news_topic_normalized.to_csv('topic_variable.csv')

In [146]:
def ESG_labeling(x):
    
    if x == 0:
        x = 'A'
    
    elif x ==  1:
        x = 'A+'
        
    elif x == 2:
        x = 'B'
    
    elif x == 3:
        x = 'B+'
        
    elif x == 4:
        x = 'C'
        
    elif x == 5:
        x = 'D'
        
    return x
    

In [147]:
#ESG 예측
news_data = pd.read_csv('topic_variable.csv')

#유의적인 변수만을 변수로 선택함

#news_variable = news_data[['company','Topic_1','Topic_2','Topic_7']] 

#전체 변수 선택 시
news_variable = news_data.drop(['date','합계','Unnamed: 0'],axis = 1)

news_variable

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,company
0,0.05,0.181461,0.119395,0.146442,0.209952,0.06664,0.148181,0.077929,무신사


In [148]:
from sklearn.preprocessing import minmax_scale

job_data = pd.DataFrame(['무신사',2.7, 2.9, 2.8, 2.5, 2.5, 2.2]).T

job_variable = job_data.iloc[0:,1:7]

job_variable = pd.concat([job_data[0],job_variable],axis=1)

job_variable.columns = ['company', 'star', 'up', 'wel', 'wl','cul','management']

job_variable

,company,star,up,wel,wl,cul,management
0,무신사,2.7,2.9,2.8,2.5,2.5,2.2


In [149]:
df_train = pd.merge(news_variable, job_variable)

df_train['company_size'] =  pd.DataFrame([416168050715,492269680950]) // 1000000000000

In [150]:
#독립변수와 종속변수 분리

x = df_train.drop(['company'], axis=1)

x

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,star,up,wel,wl,cul,management,company_size
0,0.05,0.181461,0.119395,0.146442,0.209952,0.06664,0.148181,0.077929,2.7,2.9,2.8,2.5,2.5,2.2,0


In [151]:
import joblib

model_from_joblib = joblib.load('ESG_Predict_model.pkl') 

news_topic_normalized['expect'] = model_from_joblib.predict(x)

#등급 라벨링 (여러 년도일시를 대비하여 lambda로)

news_topic_normalized['expect'] = news_topic_normalized['expect'].apply(lambda x :ESG_labeling(x))

news_topic_normalized

,date,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,합계,company,expect
0,2021,0.05,0.181461,0.119395,0.146442,0.209952,0.06664,0.148181,0.077929,1.0,무신사,B


In [152]:
news_info = pd.concat([news_topic_normalized[['date','company']],news_topic_normalized.iloc[:,1:9],news_topic_normalized[['expect']]],axis=1)

In [153]:
news_info.columns = ['date','company','취약계층 지원', '동반성장','CSR 캠페인','산업재해', '개인정보 유출',
                    '지역사회 공헌','불공정거래','상생경영','소셜 트렌드 등급']

In [154]:
news_info

,date,company,취약계층 지원,동반성장,CSR 캠페인,산업재해,개인정보 유출,지역사회 공헌,불공정거래,상생경영,소셜 트렌드 등급
0,2021,무신사,0.05,0.181461,0.119395,0.146442,0.209952,0.06664,0.148181,0.077929,B


In [155]:
print(news_info['company'][0],"소셜 트렌드",news_info.iloc[:,2:10].T.sort_values(0,ascending=False).index[:3].values)

무신사 소셜 트렌드 ['개인정보 유출' '동반성장' '불공정거래']
